# Introduction

The purpose of the tutorial is to demonstrate interactive visualization with solvation_analysis and [nglview](https://github.com/nglviewer/nglview). Please work through the basics tutorial first if you are unfamiliar with solvation_analysis.

In [37]:
# imports
import MDAnalysis as mda
from MDAnalysis import transformations
from solvation_analysis.solution import Solution
from solvation_analysis.tests import datafiles

# instantiate Universe
u = mda.Universe(datafiles.ec_emc_pdb, datafiles.ec_emc_dcd)

In [34]:
# define solute AtomGroup
li_atoms = u.atoms.select_atoms("element Li")

# define solvent AtomGroups
EC = u.residues[0:235].atoms
EMC = u.residues[235:600].atoms
PF6 = u.atoms.select_atoms("byres element P")


# instantiate solution
solution = Solution(li_atoms, 
                    {'EC': EC, 'EMC': EMC, 'PF6': PF6}, 
                    radii={'PF6': 2.6, 'EMC': 2.7, 'EC': 2.8})



solution.run()

/Users/orioncohen/projects/development/solvation-analysis/solvation_analysis/solvation_analysis/solution.py:208: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pairs[:, 1] = solvent.ix[[pairs[:, 1]]]


In [26]:
solution.radii

{'PF6': 2.6, 'EC': 2.2903880517657274, 'EMC': 2.471672152445624}

In [16]:
mda_view = nv.show_mdanalysis(u.atoms)
# mda_view.add_representation('licorice', selection='type 22', color='blue')
mda_view.display()

NGLWidget(max_frame=9)

## Primer on nglview

nglview is a powerful interactive molecular visualization package. Unfortunately, it can be a bit of hassle to get working properly. Make sure you have ipywidgets installed before working with nglview.

For more information on nglview, check out [their website](http://nglviewer.org/nglview/latest/) or the [MDAnalysis nglview tutorial](https://www.mdanalysis.org/2016/03/14/nglview/).

The cell below provides a test case for your Jupyter notebook configuration. If everything is configured properly, the cell will print a rectangle with "Hello, World" printed inside. If your Jupyter environment is configured incorrectly, it will instead print `Text(value='Hello, World')`.

In [35]:
# import nglview
import nglview as nv

# test Jupyter configuration
from ipywidgets import Text
Text("Hello, World")

Text(value='Hello, World')

## Interactive visualization

To kick off the interactive visualization workflow, we need to find something we are interested in visualizing! Let's see which atoms BN, FEC, and PF6<sub>6</sub><sup>-</sup> tend to coordinate with. We will start with BN.

First lets find out which BN solvation shell composition is most common.

In [36]:
# return the percentage of each shell
solution.speciation.speciation_percent

,EC,EMC,PF6,count
0,3,1,0,0.234426
1,2,3,0,0.178689
2,2,2,0,0.121311
3,1,3,0,0.078689
4,3,2,0,0.068852
5,4,0,0,0.057377
6,1,4,0,0.054098
7,1,2,1,0.031148
8,4,1,0,0.029508
9,1,3,1,0.027869


Looks like 5 BN is the most common shell. Let's find one to visualize!

In [36]:
# find all shells with 5 BN and nothing else
solution.speciation.find_shells({'BN': 5, 'FEC': 0, 'PF6': 0})

res_name             BN  FEC  PF6
frame solvated_atom              
0     1               5    0    0
      4               5    0    0
      7               5    0    0
      8               5    0    0
      10              5    0    0
...                  ..  ...  ...
9     38              5    0    0
      39              5    0    0
      41              5    0    0
      42              5    0    0
      48              5    0    0

[175 rows x 3 columns]

I like low numbers, so let's choose the first solvent in the zeroth frame. We'll save the AtomGroup as `shell` and just to be safe we'll print the shell DataFrame to make sure we have the right composition

In [37]:
# save the AtomGroup
bn_shell = solution.solvation_shell(1, 0)

# return the DataFrame
solution.solvation_shell(1, 0, as_df=True)

,dist,res_name,res_id
atom_id,,,
3065,2.112900,BN,256
3905,2.122193,BN,326
2057,2.138989,BN,172
1205,2.249023,BN,101
557,2.259281,BN,47


Now we are all ready to take a look! 

In [41]:
bn_shell = solution.radial_shell(1, 3)

In [42]:
mda_view = nv.show_mdanalysis(bn_shell)
mda_view.display()

NGLWidget(max_frame=9)